### Oefeningen Structural Patern Matching

Een API geeft aan welke dag van de week het is.  
De API geeft een dictionary waarvan een key in de `dict` een weekdag aanduidt.  
De API is niet consistent, de weekdag kan de volledige naam of een cijfer zijn.  

De opdracht voor deze oefening:
> Filter de API output met `match` en `case` zodat er wordt uitgeprint of het weekend is of niet.

Gegeven is de `api` functie en de `WEEK` variabele.

In [ ]:
import random

# Constant WEEK is een `tupel` van de dagen in de week. De week start op zondag.
WEEK = ('zondag', 'maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag')


def api() -> dict[str, str | int]:
    """api dat een dict teruggeeft
    structuur van de dict: { data: { weekdag: (`str` of `int`) } }
    """
    dag: str = random.choice(WEEK)
    idx: int = WEEK.index(dag)
    return {'data': {'weekdag': random.choice((dag, idx))}}

In [ ]:
# roep de api om de data te krijgen
data = api()

match data['data']:
    case {'weekdag': ('zondag' | 'zaterdag' | 0 | 6) as dag}:
        print("het is weekend")
    case {'weekdag': dag} if dag in WEEK[1:-1] or dag in range(1, 6):
        print("het is doordeweeks")
    case _:
        print("onbekende dag")

---
### off-topic: Structural Patern Matching

De Python core-ontwikkelaars dragen vrijwillig bij aan de programeertaal.  
De taal Python zelf is al meer dan 30 jaar oud, waar Python3.0.0 in 2008 verscheen.  
Het is merkwaardig dat de ontwikkelaars deze Structural Pattern Matching functionaliteit hebben geintroduceerd.  
Hieronder waarom het merkwaardig is.

In [ ]:
import re
from pathlib import PurePath

assert hasattr(re, 'match'), "err: re module has no function: 'match'"
assert hasattr(PurePath, 'match'), "err: pathlib.PurePath class has no function: 'match'"
# geen error, dan is 'match' aanwezig.

De standaard library heeft al functies met de naam _match_ in veelgebruikte modules.  
_match_ en _case_ zijn veel gebruikte woorden dus zijn zeker ook al gebruikt in third-party libraries.  
Dus door de keywords: `match` & `case` te introduceren zou de compatibiliteit met oude code gebroken moeten worden.  
Toch werkt het volgende stukje code zoals het zou moeten.

In [ ]:
# functies hebben een naam 
def case() -> str:
    return 'case'

# variabelen hebben een naam
match = case()

# twee keer match?
match match:
    # case is al een functie.
    case 'case':
        print(f"match is {match!r}")
    # twee keer case?
    case case:
        print(f"match was geen 'case' maar {case!r}")

De magie zit in de intreperter van Python.  
Als we een simpel stukje match-case code uitpakken met de module [ast] dan kan we zien wat de intreperter doet.

[ast]: https://docs.python.org/3/library/ast.html

In [ ]:
import ast

print(ast.dump(ast.parse("""
match = case = 1            # Assign
match match:                # Match
    case case:              # match_case
        print(match)        # Expr
"""), indent=2))

Hier kan worden gezien dat `match` door de intreperter wordt herkend als `Match(...)`.  
De variabele `match` wordt gezien als `id='match'`.  
Als dit geprobeerd wordt met een ander keyword zoals `def` of `class` dan is dit een `SyntaxError`.  
De code wordt dan niet geintreperteerd.  

In [ ]:
try:
    # def is een keyword die geen variable kan zijn
    exec("""def = 'def'""")
except SyntaxError as err:
    print(err)

De intreperter checkt dus voor patronen in de code waardoor `match` dus een variabele en een keyword kan zijn.  
Een _soft keyword_, en dat kan worden gevonden in de module `keyword`.  

In [ ]:
import keyword
print(f'{keyword.softkwlist = }')

assert keyword.issoftkeyword('match'), "err: match is not a 'soft keyword'"
# geen errors, dan is 'match' een soft keyword

Door dit zo voor elkaar te krijgen is oude code toch te gebruiken in nieuwe scripts waarin _match-case_ statement kan worden gebruikt.  
Het opent ook de mogenlijkheid voor andere _soft keywords_.  
Maar het kan ook zijn dat `match` en `case` soft keyword status alleen een tijdelijk is.  
Voorheen waren de `async` en `await` keywords ook soft keywords wat in [Python3.7] veranderde.  

[Python3.7]: https://docs.python.org/3.7/whatsnew/3.7.html